# Imports

In [1]:
from hyperopt import hp, tpe, STATUS_OK, Trials, fmin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import datasets

from functools import partial

import pprint

# Defining hyperparameter space

In [14]:
space = {
    'warm_start': hp.choice('warm_start', [True, False]),
    'fit_intercept': hp.choice('fit_intercept', [True, False]),
    'tol':hp.uniform('tol', 0.00001, 0.0001),
    'C': hp.uniform('C', 0.05, 2.5),
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
    'max_iter': hp.choice('max_iter', range(10, 500))
}

In [15]:
def objective(params, n_folds, X, y):
    '''Objective function for tuning logistic regression hyperparameters'''
    
    # Perform n_fold cross validation with hyperparameters
    clf = LogisticRegression(**params, random_state=42)
    scores = cross_val_score(clf, X, y, cv = n_folds, scoring = 'f1_macro')
    
    # Extract the best score
    max_score = max(scores)
    
    # Loss must be minimized
    loss = 1 - max_score
    
    #Dictionary with information for evaluation
    return {
        'loss': loss,
        'params': params,
        'status': STATUS_OK
    }

In [16]:
n_folds = 5
X , y = datasets.make_classification(n_samples = 100000, n_features = 20,
                                     n_informative=2, n_redundant=2)

train_samples = 100

X_train = X[:train_samples]
X_test = X[train_samples:]
y_train = y[:train_samples]
y_test = y[train_samples:]

In [17]:
# Trials object to track progress
trials = Trials()

# Optimize
best = fmin(
    fn=partial(objective, n_folds=n_folds, X=X_train, y=y_train),
    space=space,
    algo=tpe.suggest,
    max_evals=16,
    trials=trials
)

100%|██████████| 16/16 [00:00<00:00, 20.23trial/s, best loss: 0.051150895140664954]


c:\Users\pedro\OneDrive\Documentos\ml-engineer-with-python\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

c:\Users\pedro\OneDrive\Documentos\ml-engineer-with-python\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:

In [18]:
best

{'C': 0.1740309100987737,
 'fit_intercept': 1,
 'max_iter': 23,
 'solver': 0,
 'tol': 5.113303373592821e-05,
 'warm_start': 0}